In [ ]:
# Advanced Data Mining Project - Deliverable 2
# Regression Modeling and Performance Evaluation

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Machine learning imports
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.pipeline import Pipeline
import joblib

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")

print("=== Advanced Data Mining Project - Deliverable 2 ===")
print("Regression Modeling and Performance Evaluation")
print("=" * 60)

# ============================================================================
# STEP 1: DATA LOADING AND CLEANING (FROM DELIVERABLE 1)
# ============================================================================

print("\n1. DATA LOADING AND CLEANING")
print("-" * 50)

print("Loading UCI Online Retail Dataset...")

# Try to load UCI Online Retail Dataset
try:
    # First try local file (fastest)
    try:
        df = pd.read_excel('Online_Retail.xlsx')
        print("Loaded raw dataset from local Online_Retail.xlsx file")
        print(f"Original dataset: {len(df)} records, {len(df.columns)} features")
    except FileNotFoundError:
        # Fallback to UCI repository
        print("Local file not found. Fetching from UCI repository...")
        from ucimlrepo import fetch_ucirepo
        online_retail = fetch_ucirepo(id=352)
        df = online_retail.data.features.copy()
        print(f"Dataset loaded from UCI repository")
        print(f"Original dataset: {len(df)} records, {len(df.columns)} features")
        
except ImportError:
    print("ucimlrepo not available. Please download Online_Retail.xlsx manually")
    print("From: https://archive.ics.uci.edu/dataset/352/online+retail")
    exit()
except Exception as e:
    print(f"Error loading dataset: {e}")
    print("Please download Online_Retail.xlsx from UCI repository")
    exit()

# ============================================================================
# DATA CLEANING PIPELINE (ADAPTED FROM DELIVERABLE 1)
# ============================================================================

print("\n1.1 Data Cleaning Pipeline:")
original_shape = df.shape

# Handle Missing Values
print(f"Missing values before cleaning:")
missing_before = df.isnull().sum()
print(missing_before[missing_before > 0])

# Remove transactions without CustomerID (needed for customer analysis)
df_clean = df.dropna(subset=['CustomerID']).copy()
print(f"After removing missing CustomerID: {len(df_clean)} records")

# Remove missing descriptions if any
if df_clean['Description'].isnull().sum() > 0:
    df_clean = df_clean.dropna(subset=['Description'])
    print(f"After removing missing descriptions: {len(df_clean)} records")

# Handle Data Quality Issues
print(f"Cancellation transactions: {df_clean['InvoiceNo'].astype(str).str.startswith('C').sum()}")
print(f"Negative quantities: {(df_clean['Quantity'] < 0).sum()}")
print(f"Zero/negative prices: {(df_clean['UnitPrice'] <= 0).sum()}")

# Focus on positive transactions for regression modeling
df_clean = df_clean[(df_clean['Quantity'] > 0) & (df_clean['UnitPrice'] > 0)].copy()
print(f"After removing negative quantities and zero prices: {len(df_clean)} records")

# Convert data types
df_clean['CustomerID'] = df_clean['CustomerID'].astype(int)
df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'])
df_clean['Description'] = df_clean['Description'].str.strip().str.upper()

# Create TotalAmount
df_clean['TotalAmount'] = df_clean['Quantity'] * df_clean['UnitPrice']

# Add temporal features
df_clean['Year'] = df_clean['InvoiceDate'].dt.year
df_clean['Month'] = df_clean['InvoiceDate'].dt.month
df_clean['DayOfWeek'] = df_clean['InvoiceDate'].dt.dayofweek
df_clean['Hour'] = df_clean['InvoiceDate'].dt.hour

# Handle extreme outliers (cap at 99th percentile)
for col in ['UnitPrice', 'TotalAmount']:
    Q99 = df_clean[col].quantile(0.99)
    extreme_outliers = df_clean[col] > Q99
    if extreme_outliers.sum() > 0:
        print(f"Capping {extreme_outliers.sum()} extreme outliers in {col} at {Q99:.2f}")
        df_clean.loc[df_clean[col] > Q99, col] = Q99

print(f"\nCleaning completed: {len(df_clean)} clean records")
print(f"Data reduction: {((original_shape[0] - len(df_clean)) / original_shape[0] * 100):.1f}% records removed")

# Update main dataframe
df = df_clean.copy()

# Basic dataset overview
print(f"\nCleaned Dataset Overview:")
print(f"- Shape: {df.shape}")
print(f"- Date range: {df['InvoiceDate'].min()} to {df['InvoiceDate'].max()}")
print(f"- Unique customers: {df['CustomerID'].nunique()}")
print(f"- Unique products: {df['StockCode'].nunique()}")
print(f"- Countries: {df['Country'].nunique()}")
print(f"- Total revenue: £{df['TotalAmount'].sum():,.2f}")

# ============================================================================
# STEP 2: ADVANCED FEATURE ENGINEERING
# ============================================================================

print("\n2. ADVANCED FEATURE ENGINEERING")
print("-" * 50)

# Create a copy for feature engineering
df_features = df.copy()

# 2.1 Customer-Level Features
print("\n2.1 Creating Customer-Level Features...")

# Customer aggregation features
customer_features = df_features.groupby('CustomerID').agg({
    'InvoiceNo': 'nunique',           # Number of orders
    'Quantity': ['sum', 'mean'],      # Total and average items
    'TotalAmount': ['sum', 'mean'],   # Total and average spending
    'InvoiceDate': ['min', 'max'],    # First and last purchase
    'StockCode': 'nunique',           # Product diversity
    'Country': 'first'                # Customer country
}).round(2)

# Flatten column names
customer_features.columns = [
    'OrderCount', 'TotalQuantity', 'AvgQuantity',
    'TotalSpent', 'AvgOrderValue', 'FirstPurchase', 'LastPurchase',
    'ProductDiversity', 'Country'
]

# Calculate customer lifetime and recency
reference_date = df_features['InvoiceDate'].max()
customer_features['CustomerLifespan'] = (
    customer_features['LastPurchase'] - customer_features['FirstPurchase']
).dt.days
customer_features['Recency'] = (
    reference_date - customer_features['LastPurchase']
).dt.days

# Purchase frequency (avoid division by zero)
customer_features['PurchaseFrequency'] = customer_features['CustomerLifespan'] / customer_features['OrderCount']
customer_features['PurchaseFrequency'] = customer_features['PurchaseFrequency'].fillna(0)

print(f"Customer features created: {customer_features.shape[1]} features for {customer_features.shape[0]} customers")

# 2.2 Product-Level Features
print("\n2.2 Creating Product-Level Features...")

product_features = df_features.groupby('StockCode').agg({
    'UnitPrice': 'mean',
    'Quantity': 'mean',
    'CustomerID': 'nunique',
    'TotalAmount': 'sum'
}).round(2)

product_features.columns = ['AvgProductPrice', 'AvgProductQuantity', 'ProductPopularity', 'ProductRevenue']

print(f"Product features created: {product_features.shape[1]} features for {product_features.shape[0]} products")

# 2.3 Merge Features Back to Transaction Level
print("\n2.3 Merging Features to Transaction Level...")

# Merge customer features
df_features = df_features.merge(customer_features.reset_index(), on='CustomerID', how='left')

# Merge product features
df_features = df_features.merge(product_features, left_on='StockCode', right_index=True, how='left')

# 2.4 Create Additional Derived Features
print("\n2.4 Creating Derived Features...")

# Temporal features
df_features['IsWeekend'] = (df_features['DayOfWeek'] >= 5).astype(int)
df_features['IsBusinessHour'] = ((df_features['Hour'] >= 9) & (df_features['Hour'] <= 17)).astype(int)

# Customer engagement metric
df_features['CustomerEngagement'] = df_features['OrderCount'] * df_features['ProductDiversity']

# Price-quantity interaction
df_features['PriceQuantityInteraction'] = df_features['UnitPrice'] * df_features['Quantity']

# Customer value per order
df_features['CustomerValuePerOrder'] = df_features['TotalSpent'] / df_features['OrderCount']

# Top countries (binary feature)
top_countries = df_features['Country_x'].value_counts().head(5).index
df_features['IsTopCountry'] = df_features['Country_x'].isin(top_countries).astype(int)

# Price category
df_features['PriceCategory'] = pd.cut(df_features['UnitPrice'], 
                                     bins=[0, 2, 5, 10, float('inf')], 
                                     labels=[0, 1, 2, 3])  # Numeric categories

print(f"Total features after engineering: {df_features.shape[1]}")

# ============================================================================
# STEP 3: REGRESSION DATASET PREPARATION
# ============================================================================

print("\n3. REGRESSION DATASET PREPARATION")
print("-" * 50)

# Select features for modeling (focus on numerical features that work well for regression)
feature_columns = [
    # Basic transaction features
    'Quantity', 'UnitPrice', 'Hour', 'DayOfWeek', 'Month',
    # Temporal features  
    'IsWeekend', 'IsBusinessHour',
    # Customer features
    'OrderCount', 'AvgOrderValue', 'ProductDiversity', 'Recency', 
    'PurchaseFrequency', 'CustomerEngagement', 'CustomerValuePerOrder',
    # Product features
    'AvgProductPrice', 'AvgProductQuantity', 'ProductPopularity',
    # Derived features
    'IsTopCountry', 'PriceCategory'
]

print(f"Selected features for modeling: {len(feature_columns)}")
for i, feature in enumerate(feature_columns, 1):
    print(f"  {i:2d}. {feature}")

# Prepare regression dataset
X = df_features[feature_columns].copy()
y = df_features['TotalAmount'].copy()

# Handle any remaining missing values
print(f"\nMissing values in features:")
missing_features = X.isnull().sum()
if missing_features.sum() > 0:
    print(missing_features[missing_features > 0])
    X = X.fillna(X.median())
    print("Missing values filled with median")
else:
    print("No missing values found")

# Remove any infinite values
X = X.replace([np.inf, -np.inf], np.nan)
if X.isnull().sum().sum() > 0:
    X = X.fillna(X.median())
    print("Infinite values replaced")

print(f"\nFinal dataset for modeling:")
print(f"- Features: {X.shape[1]}")
print(f"- Samples: {X.shape[0]:,}")
print(f"- Target range: £{y.min():.2f} to £{y.max():.2f}")
print(f"- Target mean: £{y.mean():.2f}")

# ============================================================================
# STEP 4: DATA SPLITTING AND PREPROCESSING  
# ============================================================================

print("\n4. DATA SPLITTING AND PREPROCESSING")
print("-" * 50)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)

print(f"Training set: {X_train.shape[0]:,} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Test set: {X_test.shape[0]:,} samples ({X_test.shape[0]/len(X)*100:.1f}%)")

# Feature scaling for regularized models
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Store feature names
feature_names = X.columns.tolist()

print("Feature scaling completed for regularized models")

# ============================================================================
# STEP 5: REGRESSION MODEL DEVELOPMENT
# ============================================================================

print("\n5. REGRESSION MODEL DEVELOPMENT")
print("-" * 50)

# Initialize storage
model_results = {}
models = {}

# Evaluation function
def evaluate_model(name, model, X_train_data, X_test_data, y_train, y_test, use_cv=True):
    """Comprehensive model evaluation"""
    
    # Fit model
    print(f"\nTraining {name}...")
    model.fit(X_train_data, y_train)
    
    # Predictions
    y_train_pred = model.predict(X_train_data)
    y_test_pred = model.predict(X_test_data)
    
    # Metrics
    results = {
        'train_r2': r2_score(y_train, y_train_pred),
        'test_r2': r2_score(y_test, y_test_pred),
        'train_rmse': np.sqrt(mean_squared_error(y_train, y_train_pred)),
        'test_rmse': np.sqrt(mean_squared_error(y_test, y_test_pred)),
        'train_mae': mean_absolute_error(y_train, y_train_pred),
        'test_mae': mean_absolute_error(y_test, y_test_pred)
    }
    
    # Cross-validation
    if use_cv:
        cv_scores = cross_val_score(model, X_train_data, y_train, cv=5, scoring='r2', n_jobs=-1)
        results['cv_r2_mean'] = cv_scores.mean()
        results['cv_r2_std'] = cv_scores.std()
    else:
        results['cv_r2_mean'] = results['test_r2']  # Fallback
        results['cv_r2_std'] = 0.0
    
    # Store results
    model_results[name] = results
    models[name] = model
    
    print(f"{name} Results:")
    print(f"  Test R²: {results['test_r2']:.4f}")
    print(f"  Test RMSE: £{results['test_rmse']:.2f}")
    print(f"  Test MAE: £{results['test_mae']:.2f}")
    if use_cv:
        print(f"  CV R²: {results['cv_r2_mean']:.4f} ± {results['cv_r2_std']:.4f}")
    
    return results, model

# 5.1 Linear Regression (Baseline)
linear_model = LinearRegression()
linear_results, _ = evaluate_model("Linear Regression", linear_model, X_train, X_test, y_train, y_test)

# 5.2 Ridge Regression
print("\n5.2 Ridge Regression with Hyperparameter Tuning")
ridge_params = {'alpha': [0.1, 1.0, 10.0, 100.0]}
ridge_grid = GridSearchCV(Ridge(random_state=42), ridge_params, cv=3, scoring='r2', n_jobs=-1)
ridge_grid.fit(X_train_scaled, y_train)

print(f"Best Ridge alpha: {ridge_grid.best_params_['alpha']}")
best_ridge = Ridge(alpha=ridge_grid.best_params_['alpha'], random_state=42)
ridge_results, _ = evaluate_model("Ridge Regression", best_ridge, X_train_scaled, X_test_scaled, y_train, y_test)

# 5.3 Lasso Regression  
print("\n5.3 Lasso Regression with Hyperparameter Tuning")
lasso_params = {'alpha': [0.1, 1.0, 10.0]}
lasso_grid = GridSearchCV(Lasso(random_state=42, max_iter=2000), lasso_params, cv=3, scoring='r2', n_jobs=-1)
lasso_grid.fit(X_train_scaled, y_train)

print(f"Best Lasso alpha: {lasso_grid.best_params_['alpha']}")
best_lasso = Lasso(alpha=lasso_grid.best_params_['alpha'], random_state=42, max_iter=2000)
lasso_results, _ = evaluate_model("Lasso Regression", best_lasso, X_train_scaled, X_test_scaled, y_train, y_test)

# ============================================================================
# STEP 6: MODEL COMPARISON AND EVALUATION
# ============================================================================

print("\n6. MODEL COMPARISON AND EVALUATION") 
print("-" * 50)

# Create comparison table
comparison_df = pd.DataFrame(model_results).T
comparison_df = comparison_df[['test_r2', 'test_rmse', 'test_mae', 'cv_r2_mean', 'cv_r2_std']]
comparison_df.columns = ['Test_R2', 'Test_RMSE', 'Test_MAE', 'CV_R2_Mean', 'CV_R2_Std']
comparison_df = comparison_df.round(4)

print("\n6.1 Model Performance Summary:")
print("="*70)
print(f"{'Model':<20} {'Test R²':<10} {'RMSE':<10} {'MAE':<10} {'CV R²':<15}")
print("="*70)

for model_name, row in comparison_df.iterrows():
    print(f"{model_name:<20} {row['Test_R2']:<10.4f} £{row['Test_RMSE']:<9.2f} £{row['Test_MAE']:<9.2f} {row['CV_R2_Mean']:.3f}±{row['CV_R2_Std']:.3f}")

print("="*70)

# Find best model
best_model_name = comparison_df['Test_R2'].idxmax()
best_r2 = comparison_df.loc[best_model_name, 'Test_R2']
best_rmse = comparison_df.loc[best_model_name, 'Test_RMSE']

print(f"\nBEST MODEL: {best_model_name}")
print(f"   Test R²: {best_r2:.4f} (explains {best_r2*100:.1f}% of variance)")
print(f"   Test RMSE: £{best_rmse:.2f} (average prediction error)")

# ============================================================================
# STEP 7: VISUALIZATIONS
# ============================================================================

print("\n7. MODEL VISUALIZATION AND ANALYSIS")
print("-" * 50)

# Create visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 7.1 Model Performance Comparison
model_names = comparison_df.index
r2_scores = comparison_df['Test_R2'].values
rmse_scores = comparison_df['Test_RMSE'].values

axes[0, 0].bar(model_names, r2_scores, color='skyblue', alpha=0.8)
axes[0, 0].set_title('Model Comparison: Test R²', fontsize=14, fontweight='bold')
axes[0, 0].set_ylabel('R² Score')
axes[0, 0].tick_params(axis='x', rotation=45)
axes[0, 0].grid(True, alpha=0.3)
for i, v in enumerate(r2_scores):
    axes[0, 0].text(i, v + 0.01, f'{v:.3f}', ha='center', fontweight='bold')

axes[0, 1].bar(model_names, rmse_scores, color='lightcoral', alpha=0.8)
axes[0, 1].set_title('Model Comparison: Test RMSE', fontsize=14, fontweight='bold')
axes[0, 1].set_ylabel('RMSE (£)')
axes[0, 1].tick_params(axis='x', rotation=45)
axes[0, 1].grid(True, alpha=0.3)

# 7.2 Feature Importance (Ridge Coefficients)
if 'Ridge Regression' in models:
    ridge_model = models['Ridge Regression']
    feature_importance = pd.DataFrame({
        'Feature': feature_names,
        'Importance': np.abs(ridge_model.coef_)
    }).sort_values('Importance', ascending=True)
    
    top_features = feature_importance.tail(10)
    axes[1, 0].barh(top_features['Feature'], top_features['Importance'], color='lightgreen', alpha=0.8)
    axes[1, 0].set_title('Top 10 Feature Importance (Ridge)', fontsize=14, fontweight='bold')
    axes[1, 0].set_xlabel('Absolute Coefficient Value')
    axes[1, 0].grid(True, alpha=0.3)

# 7.3 Actual vs Predicted for Best Model
best_model_obj = models[best_model_name]
if best_model_name == "Linear Regression":
    y_pred = best_model_obj.predict(X_test)
elif best_model_name == "Random Forest":
    y_pred = best_model_obj.predict(X_test) 
else:
    y_pred = best_model_obj.predict(X_test_scaled)

axes[1, 1].scatter(y_test, y_pred, alpha=0.6, color='blue')
axes[1, 1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[1, 1].set_xlabel('Actual TotalAmount (£)')
axes[1, 1].set_ylabel('Predicted TotalAmount (£)')
axes[1, 1].set_title(f'{best_model_name}: Actual vs Predicted\nR² = {best_r2:.3f}', 
                     fontsize=14, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# ============================================================================
# STEP 8: FEATURE ANALYSIS
# ============================================================================

print("\n8. FEATURE IMPORTANCE ANALYSIS")
print("-" * 50)

if 'Ridge Regression' in models:
    ridge_model = models['Ridge Regression']
    feature_analysis = pd.DataFrame({
        'Feature': feature_names,
        'Coefficient': ridge_model.coef_,
        'Abs_Coefficient': np.abs(ridge_model.coef_)
    }).sort_values('Abs_Coefficient', ascending=False)
    
    print("\n8.1 Top 10 Most Important Features (Ridge Regression):")
    print("-" * 60)
    for i, (_, row) in enumerate(feature_analysis.head(10).iterrows(), 1):
        direction = "increases" if row['Coefficient'] > 0 else "decreases"
        print(f"{i:2d}. {row['Feature']:<25} {direction:<10} {row['Coefficient']:+8.4f}")

if 'Lasso Regression' in models:
    lasso_model = models['Lasso Regression']
    lasso_features = pd.DataFrame({
        'Feature': feature_names,
        'Coefficient': lasso_model.coef_
    })
    selected_features = lasso_features[lasso_features['Coefficient'] != 0].sort_values('Coefficient', key=abs, ascending=False)
    
    print(f"\n8.2 Lasso Feature Selection:")
    print(f"Selected {len(selected_features)} out of {len(feature_names)} features")
    if len(selected_features) > 0:
        print("\nSelected features:")
        for i, (_, row) in enumerate(selected_features.head(10).iterrows(), 1):
            direction = "increases" if row['Coefficient'] > 0 else "decreases"
            print(f"{i:2d}. {row['Feature']:<25} {direction:<10} {row['Coefficient']:+8.4f}")

# ============================================================================
# STEP 9: BUSINESS INSIGHTS AND RECOMMENDATIONS  
# ============================================================================

print("\n9. BUSINESS INSIGHTS AND RECOMMENDATIONS")
print("-" * 50)

print(f"\n9.1 Model Performance Summary:")
print(f"Best Model: {best_model_name}")
print(f"Prediction Accuracy: {best_r2:.1%} of variance explained")
print(f"Average Prediction Error: £{best_rmse:.2f}")
print(f"Business Impact: Model can predict transaction values for revenue forecasting")

print(f"\n9.2 Key Predictive Features:")
if 'Ridge Regression' in models:
    top_3_features = feature_analysis.head(3)
    for i, (_, row) in enumerate(top_3_features.iterrows(), 1):
        impact = "increases" if row['Coefficient'] > 0 else "decreases"
        print(f"{i}. {row['Feature']}: {impact} transaction value (coef: {row['Coefficient']:+.4f})")

print(f"9.3 Business Recommendations:")
print(f"• Revenue Forecasting: Use model for monthly/quarterly revenue predictions")
print(f"• Customer Targeting: Focus on high-value customer segments identified")  
print(f"• Pricing Strategy: Leverage price-quantity relationships discovered")
print(f"• Inventory Planning: Prioritize products with high predictive importance")
print(f"• Marketing Timing: Optimize campaigns based on temporal patterns found")

print(f"\n9.4 Model Deployment Readiness:")
print(f"• Prediction Range: £{y_test.min():.2f} - £{y_test.max():.2f}")
print(f"• Expected Accuracy: ±£{best_rmse:.2f} on average")

# ============================================================================
# STEP 10: MODEL PERSISTENCE
# ============================================================================

print("\n10. SAVING TRAINED MODELS")
print("-" * 50)

# Save the best model and preprocessing components
try:
    joblib.dump(models[best_model_name], 'best_regression_model.pkl')
    joblib.dump(scaler, 'feature_scaler.pkl')
    
    # Save model metadata
    model_metadata = {
        'best_model': best_model_name,
        'feature_names': feature_names,
        'performance': {
            'test_r2': float(best_r2),
            'test_rmse': float(best_rmse),
            'cv_r2_mean': float(comparison_df.loc[best_model_name, 'CV_R2_Mean']),
            'cv_r2_std': float(comparison_df.loc[best_model_name, 'CV_R2_Std'])
        },
        'feature_importance': feature_analysis.head(10).to_dict('records') if 'Ridge Regression' in models else []
    }
    
    import json
    with open('model_metadata.json', 'w') as f:
        json.dump(model_metadata, f, indent=2)
    
    print("Model artifacts saved successfully:")
    print("   • best_regression_model.pkl - Trained model")  
    print("   • feature_scaler.pkl - Feature preprocessing")
    print("   • model_metadata.json - Performance metrics")
    
except Exception as e:
    print(f"Error saving model files: {e}")

# ============================================================================
# FINAL SUMMARY
# ============================================================================

print(f"\n" + "="*60)
print(f"PROJECT DELIVERABLE 2 COMPLETED SUCCESSFULLY!")
print(f"="*60)
print(f"Dataset: {len(df):,} transactions processed and cleaned")
print(f"Features: {len(feature_names)} engineered features created") 
print(f"Models: {len(models)} regression models trained and evaluated")
print(f"Best Model: {best_model_name} (R² = {best_r2:.3f})")
print(f"="*60)

